In [2]:
### Benjamin Tollison ###
import sympy as sp
from IPython.display import display, Math, Latex
from sympy.solvers.solveset import linsolve
from sympy.solvers.pde import pdsolve
from sympy import  lambdify, Matrix,sin,cos, sinh, cosh, exp, pi, symbols, integrate, Function, Eq
import numpy as np
import matplotlib.pyplot as plt
import itertools
def displayEquations(LHS,RHS):
    left = sp.latex(LHS)
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
    np.set_printoptions(suppress=True)

In [3]:
# optional
def bmatrix(a):
    """Returns a LaTeX bmatrix

    :a: numpy array
    :returns: LaTeX bmatrix as a string
    """
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    return '\n'.join(rv)

In [4]:
# a
x1,x2,x3,x4,x5,x6 = symbols('x_1,x_2,x_3,x_4,x_5,x_6')
F1,F2,F3,M1,M2,M3 = symbols('F_1,F_2,F_3,M_1,M_2,M_3')
x = Matrix([x1,x2,x3,x4,x5,x6])
b = Matrix([F1,F2,F3,M1,M2,M3])

In [5]:
Thrust_location = np.array([
  [2,-8,-8],
  [-8,-2,-8],
  [8,-8,-2],
  [8,2,8],
  [-8,8,2],
  [-2,8,8]
])
Thrust_direction = np.array([
  [-1,0,0],
  [0,1,0],
  [0,0,1],
  [0,-1,0],
  [0,0,-1],
  [1,0,0]
])
# Forces
A = np.zeros(shape=(6,6))
for row in range(3):
  for column in range(6):
    A[row][column] = Thrust_direction.T[row][column]
# M_x
for thrust_number in range(6):
  r = Thrust_location[thrust_number][1:]
  F = Thrust_direction[thrust_number][1:]
  A[3][thrust_number] = np.cross(r,F)
# M_y
for thrust_number in range(6):
  r = Thrust_location[thrust_number][:]
  r = np.delete(r,1,0)
  F = Thrust_direction[thrust_number][:]
  F = np.delete(F,1,0)
  A[4][thrust_number] = np.cross(r,F)
# M_z
for thrust_number in range(6):
  r = Thrust_location[thrust_number][:2]
  F = Thrust_direction[thrust_number][:2]
  A[5][thrust_number] = np.cross(r,F)
print(A)
# displayEquations('A',A)
bmatrix(A)

[[-1.  0.  0.  0.  0.  1.]
 [ 0.  1.  0. -1.  0.  0.]
 [ 0.  0.  1.  0. -1.  0.]
 [ 0.  8. -8.  8. -8.  0.]
 [-8.  0.  8.  0.  8. -8.]
 [-8. -8.  0. -8.  0. -8.]]


'\\begin{bmatrix}\n  -1. & 0. & 0. & 0. & 0. & 1.\\\\\n  0. & 1. & 0. & -1. & 0. & 0.\\\\\n  0. & 0. & 1. & 0. & -1. & 0.\\\\\n  0. & 8. & -8. & 8. & -8. & 0.\\\\\n  -8. & 0. & 8. & 0. & 8. & -8.\\\\\n  -8. & -8. & 0. & -8. & 0. & -8.\\\\\n\\end{bmatrix}'

In [6]:
# b
x = np.array([1,1,0,0,1,0])
b = np.matmul(A,x)
displayEquations('b',b)
bmatrix(b)

<IPython.core.display.Math object>

'\\begin{bmatrix}\n  -1. & 1. & -1. & 0. & 0. & -16.\\\\\n\\end{bmatrix}'

In [7]:
# c
moment_only_combinations = []
thrust_combinations = list(itertools.product([0, 1], repeat=6))
for instance in thrust_combinations:
  x = instance
  b = np.matmul(A,x)
  if b[0] == 0 and b[1] == 0 and b[2] == 0:
    moment_only_combinations.append(instance)
print(moment_only_combinations)
for i in moment_only_combinations:
  print(np.matmul(A,np.array(i)))
print(len(thrust_combinations))

[(0, 0, 0, 0, 0, 0), (0, 0, 1, 0, 1, 0), (0, 1, 0, 1, 0, 0), (0, 1, 1, 1, 1, 0), (1, 0, 0, 0, 0, 1), (1, 0, 1, 0, 1, 1), (1, 1, 0, 1, 0, 1), (1, 1, 1, 1, 1, 1)]
[0. 0. 0. 0. 0. 0.]
[  0.   0.   0. -16.  16.   0.]
[  0.   0.   0.  16.   0. -16.]
[  0.   0.   0.   0.  16. -16.]
[  0.   0.   0.   0. -16. -16.]
[  0.   0.   0. -16.   0. -16.]
[  0.   0.   0.  16. -16. -32.]
[  0.   0.   0.   0.   0. -32.]
64


In [8]:
x_moments_only = []
